# Answer to Question3(b)

#### This is based on the same intuition as before that we can model using the data where the company has granted the loans.
#### The biggest problem here is the imbalance in the dataset because majority of the loans that were granted were for 360 units.
#### We could try using down sampling or SMOTE but there are no major records other than that of 360 units and it would have heavily impacted the accuracy. 
#### Since there are only 9 classes in 'Term Loan Amount' its is treated here as a classification probelm but in general it should be a regression problem. 
#### Same as before the first model would predict if a customer is eligible for the loan or not and the second and the thirdmodel could give a different amount or timeline that the company could offer. 
#### The best thing to do here in regards to solving is issue with 'Term Loan Amount' would be to get more data that belong to different classes.

In [1]:
from azureml import Workspace

ws = Workspace()
ds = ws.datasets['processed-loan.csv']
df = ds.to_dataframe()

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [4]:
df['Loan_Amount_Term'].value_counts()

360.0    359
180.0     29
300.0      8
480.0      6
84.0       3
240.0      3
120.0      3
60.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [5]:
#categorical
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df['Married'].fillna(df['Married'].mode()[0], inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
#numerical
df['LoanAmount'].fillna(df['LoanAmount'].mean(), inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder
category = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area','Loan_Status'] 
encoder = LabelEncoder()
for i in category:
  df[i] = encoder.fit_transform(df[i])
df.dtypes

Loan_ID               object
Gender                 int64
Married                int64
Dependents             int64
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
Loan_Status            int64
dtype: object

In [8]:
df.drop(['Loan_ID','Loan_Status'],axis=1,inplace=True)

In [9]:
df['TotalIncome']= df['ApplicantIncome'] +df['CoapplicantIncome'] 


In [10]:
df.drop(['ApplicantIncome','CoapplicantIncome'],axis=1,inplace=True)

In [11]:
df.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'TotalIncome'],
      dtype='object')

In [12]:
y = df.pop('Loan_Amount_Term')

In [13]:
x = df

In [18]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=4)

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1235)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((337, 9), (85, 9), (337,), (85,))

In [19]:
clf.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [21]:
yhat = clf.predict(x_test)

In [23]:
clf.score(x_train,y_train)

0.87240356083086057

In [24]:
clf.score(x_test,y_test)

0.90588235294117647

In [26]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(yhat,y_test))
print(classification_report(yhat,y_test))

[[ 0  0  0  0]
 [ 0  0  0  0]
 [ 5  1 77  2]
 [ 0  0  0  0]]
             precision    recall  f1-score   support

      180.0       0.00      0.00      0.00         0
      300.0       0.00      0.00      0.00         0
      360.0       1.00      0.91      0.95        85
      480.0       0.00      0.00      0.00         0

avg / total       1.00      0.91      0.95        85

